In [1]:
# code written by Yash Vardhan

import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score
from keras.models import Model
from keras.layers import (
    Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout,
    BatchNormalization, Activation, Bidirectional, Multiply,
    Permute, Lambda, GlobalAveragePooling1D, Softmax
)
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint



In [2]:
df = pd.read_csv("this_data_set.csv")

# Convert ECG string to list
df['ecg'] = df['ecg'].apply(ast.literal_eval)

X = pad_sequences(df['ecg'], padding='post', dtype='float32')
y = df['type'].values


In [3]:
X_flat = X.reshape(-1, 1)
scaler = StandardScaler()
X_scaled_flat = scaler.fit_transform(X_flat)
X = X_scaled_flat.reshape(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)




In [4]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train
)
class_weights_dict = dict(enumerate(class_weights))

input_layer = Input(shape=(X_train.shape[1], 1))

x = Conv1D(64, 5, padding='same')(input_layer)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling1D(2)(x)

x = Conv1D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Bidirectional(LSTM(64))(x)

x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)


In [5]:
model = Model(inputs=input_layer, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.3,
    class_weight=class_weights_dict,
    callbacks=[early_stop, checkpoint]
)

y_pred = (model.predict(X_test) > 0.5).astype("int32")


print("\nAccuracy:", accuracy_score(y_test, y_pred) * 100)

Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 68s/step - accuracy: 0.4474 - loss: 0.6975  

KeyboardInterrupt: 